In [ ]:
from sklearn.impute import KNNImputer
from tqdm.notebook import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV 

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
class serial_impute():
  
  def __init__(self,data):
    self.data=data
    self.amt=data[['REG_YYMM', 'STD_CLSS_NM','CARD_SIDO_NM','CSTMR_CNT','AMT','CNT']].groupby(['REG_YYMM', 'STD_CLSS_NM','CARD_SIDO_NM'],as_index=False).sum()
    self.sido_list = list(set(data['CARD_SIDO_NM']))
    self.std_list=list(set(data['STD_CLSS_NM']))
    self.month_list = list(set(data['REG_YYMM']))

#데이터 프레임 조작해서 원하는 형식으로 맞춰주기
  def pd_maker(self,sido,std):
    p_amt=self.amt[(self.amt['CARD_SIDO_NM']==sido)&(self.amt['STD_CLSS_NM']==std)]
    p_val=p_amt[['REG_YYMM','CSTMR_CNT','AMT']]
    p_val.replace(0,np.nan,inplace=True)
    p_val['MM']=p_val['REG_YYMM']%100
    p_val['YY']=p_val['REG_YYMM']//100
    p_val['MM']=p_val['MM'].astype(int)
    p_val['YY']=p_val['YY'].astype(int)
    amt_final=p_val[['YY','MM',"CSTMR_CNT",'AMT']]
    amt_final.reset_index(inplace=True)
    return amt_final

#결측치 채워넣기 by sum(AMT) data
#원 데이터에서 submission 파일과 겹치는 칼럼 외의 칼럼들을 제거한 후 AMT를 더해주니
#도시, 업종별로 값이 존재하지 않는 데이터가 존재.
#비식별화 조치로 없어진 데이터는 추후에도 없어질 가능성이 높다 판단되 결측치 처리를 해줌.
  def full_imputer(self):
    result_pd=pd.DataFrame(columns=['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])
    for sinan in tqdm(self.sido_list):
      for stnan in tqdm(self.std_list):
        sp_data=self.data[(self.data['CARD_SIDO_NM']==sinan)&(self.data['STD_CLSS_NM']==stnan)]    
        #위에서 만든 형태 지정 메소드 사용.
        amt_pd=self.pd_maker(sinan,stnan)
        #상관계수.
        d_corr=sp_data.corr()
        ca_corr=d_corr['CSTMR_CNT']['AMT']
        #값이 결측된 행이 10개 미만인 데이터에서만 impute 실행.
        if 0<amt_pd['AMT'].isnull().sum() <10:
        #상관계수가 0.66 이상인 데이터에 한해서 KNNImputer를 사용해 예측 실행.
        #CSTMR_CNT와 AMT의 전반적인 상관계수가 높기 때문 + 상관계수가 높아 예측값의 정확도도 높을 것이라 보기 때문.        
          if ca_corr>0.66:
            imputer = KNNImputer(n_neighbors=3, weights="uniform")
            nn_pd=pd.DataFrame(imputer.fit_transform(amt_pd))
            nn_pd.columns=amt_pd.columns
            nn_pd['CARD_SIDO_NM']=str(sinan)
            nn_pd['STD_CLSS_NM']=str(stnan)
            result_pd=pd.concat([result_pd,nn_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
          else:
        #울산 면세점, 전북 면세점 같은 경우는 값이 치우쳐짐 + 최근 데이터 결측이 심해 0으로 채워줌.
            if ((sinan=='울산')&(stnan=='면세점'))|((sinan=='전북')&(stnan=='내항 여객 운송업')):
              amt_pd.replace(np.nan,0,inplace=True)
              amt_pd['CARD_SIDO_NM']=str(sinan)
              amt_pd['STD_CLSS_NM']=str(stnan)
              result_pd=pd.concat([result_pd,amt_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
            else:
        #그 외의 데이터는 Iterativeimputer의 BayesianRidge를 활용해 결측치 처리.
        #median 값을 사용.
              cc_pd=amt_pd[['YY','MM','AMT']]
              nn_pd = pd.DataFrame(IterativeImputer(random_state=8,initial_strategy='median',estimator=BayesianRidge()).fit_transform(cc_pd.values))
              nn_pd.columns=cc_pd.columns
              nn_pd['CARD_SIDO_NM']=str(sinan)
              nn_pd['STD_CLSS_NM']=str(stnan)
              result_pd=pd.concat([result_pd,nn_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
        #결측된 행이 10개 이상이면 비식별화 확률이 높다 판단하여 모두 0으로 채워주었다.
        else:
          amt_pd.replace(np.nan,0,inplace=True)
          amt_pd['CARD_SIDO_NM']=str(sinan)
          amt_pd['STD_CLSS_NM']=str(stnan)
          result_pd=pd.concat([result_pd,amt_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
    result_pd=result_pd.reset_index()
  
    return result_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
data_jj=pd.read_csv('C:/Users/kzvxy/conda_py/jeju_data_ver1/201901-202003.csv')

In [ ]:
#sub 데이터 가져오기
sub=pd.read_csv('C:/Users/kzvxy/conda_py/jeju_data_ver1/submission.csv')

In [ ]:
#04 월 데이터
data_04=pd.read_csv('C:/Users/kzvxy/conda_py/jeju_data_ver1/202004.csv')

In [ ]:
full_nn=pd.read_csv('C:/Users/kzvxy/conda_py/full_nn.csv')
full_nn['YY']=full_nn['YY'].astype(int)
full_nn['MM']=full_nn['MM'].astype(int)
full_nn=full_nn[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
d04_pd=data_04[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']].groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).agg(['sum']).reset_index()
d04_pd.columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']
d04_pd['MM']=d04_pd['REG_YYMM']%100
d04_pd['YY']=d04_pd['REG_YYMM']//100
d04_pd['MM']=d04_pd['MM'].astype(int)
d04_pd['YY']=d04_pd['YY'].astype(int)
d04_pd=d04_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
full_n4=pd.concat([full_nn,d04_pd])

In [ ]:
class x_y_maker():
  def __init__(self,data):
    self.data=data
    #데이터 카피
    self.ori_data=data.copy()
    #AGE->AGE_GROUP
    #시도간 이동 라벨링
    self.encoder_s=LabelEncoder()
    self.encoder_c=LabelEncoder()
    self.stscaler=StandardScaler()

#특성 더 넣고 싶은 게 있으면 여기서 칼럼 만들어주면 된다.
#잘 모르겠으면 그냥 물어봐줘...
  
  def col_setting(self,full_data):
    #STD_CLSS_LB 만들기 (업종 데이터 라벨화)
    sample_label_s=self.encoder_s.transform(full_data['STD_CLSS_NM'])
    full_data['STD_CLSS_LB']=sample_label_s
    #CARD_SIDO_LB 만들기 (시도 데이터 라벨화)
    sample_label_c=self.encoder_c.transform(full_data['CARD_SIDO_NM'])
    full_data['CARD_SIDO_LB']=sample_label_c
    # MM, YY 나눠주기
    #필요한 칼럼만 축소
    actual_data=full_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','AMT']]
    
    return actual_data 

    # 원데이터 변형
  def data_maker(self):
    # CARD_CCG_NM, HOM_SIDO_NM, HOM_CCG_NM, SEX_CTGO, AGE, AGE_GROUP, CSTMR_CNT, CNT 삭제
    # CSTMR_CNT, CNT를 SUB에 반영할 수가 없다. ㅠ
    # 원데이터에서는 따로 NaN을 집어넣어주지 않음. NaN 때문에 학습이 안되서.
    # 만약 결측치 처리한 게 있으면 원데이터 대신 넣어주면 된다.
    mak_data=self.ori_data[['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM',
                            'AMT']].groupby(['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).agg(['sum']).reset_index()
    mak_data.columns=['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM','AMT']
    #라벨 인코딩
    self.encoder_s.fit(mak_data['STD_CLSS_NM'])
    self.encoder_c.fit(mak_data['CARD_SIDO_NM'])
    t_data=self.col_setting(mak_data)
    X_t=t_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB']]
    y_ft=t_data[['AMT']]
    #shape 맞춰주기
    y_ts=self.stscaler.fit_transform(np.log1p(y_ft))
    y_t=y_ts.flatten()
    return X_t, y_t

    # sub데이터 변형
  def sub_maker(self,subb):
    sub=subb.copy()
    sub['YY']=sub['REG_YYMM']//100
    sub['MM']=sub['REG_YYMM']%100
    sub['ATM']=np.nan
    sub_data=self.col_setting(sub)
    X_s=sub_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB']]
    return X_s

    # sub데이터 디코드, sub_t에는 서브미션에서 AMT제외한 거, sub_p에는 predict value 넣어주면 됨.
  def sub_decoder(self,sub_tt,sub_p):
    sub_t=sub_tt.copy()
    sub_t=sub_t[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB']]
    sub_t['STD_CLSS_NM']=self.encoder_s.inverse_transform(sub_t['STD_CLSS_LB'])
    sub_t['CARD_SIDO_NM']=self.encoder_c.inverse_transform(sub_t['CARD_SIDO_LB'])
    sub_t['REG_YYMM']=sub_t['YY']*100+sub_t['MM']
    st_p=self.stscaler.inverse_transform(sub_p)
    fin_p= np.expm1(st_p)
    sub_t['AMT']=np.round(fin_p,0)
    sub_t['AMT']=np.where(sub_t['AMT']<1,0,sub_t['AMT'])
    sub_t=sub_t[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
    return sub_t

In [ ]:
xy_maker = x_y_maker(full_n4)

In [ ]:
# 객체 내 메서드 실행 => 원 데이터 변형
x,y=xy_maker.data_maker()

In [ ]:
# 객체 내 메서드 실행 => sub 데이터 변형
X_sub=xy_maker.sub_maker(sub)

In [ ]:
# 트레인, 테스트 셋 분리 random_state=777
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=777,shuffle = True)

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
#xgb2 파라미터 튜닝에서 가장 좋은 파라미터 나온 것
xgb3=xgb.XGBRegressor(alpha=0.02,colsample_bytree=0.7,learning_rate=0.05,max_depth=10,
                      min_child_weight=2,n_estimators=2000,random_state=545,objective='reg:squarederror',
                     subsample=0.7,reg_lambda=0)

In [ ]:
#Train 데이터가 아닌 원 데이터 전체 학습
xgb3.fit(x,y)

XGBRegressor(alpha=0.02, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=545,
             reg_alpha=0.0199999996, reg_lambda=0, scale_pos_weight=1,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
xgb3.predict(X_sub)

array([-0.02390462, -0.04092395,  0.54379725, ...,  0.22735143,
       -0.20206267, -0.04358119], dtype=float32)

In [ ]:
submiss3=xy_maker.sub_decoder(X_sub,xgb3.predict(X_sub))
submiss3.index.name='id'

In [ ]:
submiss3

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.200353e+07
1,202007,강원,건강보조식품 소매업,7.364081e+07
2,202004,강원,골프장 운영업,2.965225e+09
3,202007,강원,골프장 운영업,3.388222e+09
4,202004,강원,과실 및 채소 소매업,8.114225e+08
...,...,...,...,...
1389,202007,충북,호텔업,1.100675e+07
1390,202004,충북,화장품 및 방향제 소매업,4.717178e+08
1391,202007,충북,화장품 및 방향제 소매업,4.013088e+08


In [ ]:
#학습결과 저장
submiss3.to_csv('sub_amtonly_xgb_cls.csv', encoding='utf-8-sig')

In [ ]:
#19년 데이터만으로 학습시켜보겠음.
fu19=full_nn[full_nn['YY']==2019]

In [ ]:
xy_maker = x_y_maker(fu19)

In [ ]:
x19,y19=xy_maker.data_maker()
X_sub, y_sub=xy_maker.sub_maker(sub)

In [ ]:
#xgb 1~3과 마찬가지로 파라미터 튜닝에서 가장 좋은 값나온 것
xgb4=xgb.XGBRegressor(alpha=0.02,colsample_bytree=0.7,learning_rate=0.05,max_depth=10,
                      min_child_weight=2,n_estimators=2000,random_state=545,objective='reg:squarederror',
                     subsample=0.7,reg_lambda=0)

In [ ]:
xgb4.fit(x19,y19)

XGBRegressor(alpha=0.02, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=545,
             reg_alpha=0.0199999996, reg_lambda=0, scale_pos_weight=1,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
submiss4=xy_maker.sub_decoder(X_sub,xgb4.predict(X_sub))
submiss4.index.name='id'

In [ ]:
submiss4.to_csv('sub_amtonly19_xgb.csv', encoding='utf-8-sig')

In [ ]:
# 그리드 서치
xgbf = xgb.XGBRegressor()
parameters= {'booster':['gbtree'],
              'learning_rate': [0.01,0.03,0.05], #so called `eta` value
              'max_depth': [4,5,6,7,8,9,10],
              'objective':['reg:squarederror'],
              'min_child_weight': [2,3,4,5,6,7,8],
              'subsample': [0.7],
              'reg_lambda':[0,0.01,0.02,0.03,0.05],
              'alpha':[0,0.01,0.05,0.1,0.2],
              'colsample_bytree': [0.7],
              'gamma':[0,0.01,0.02,0.05,0.1],
              'n_estimators': [2000],
              'random_state':[545]}

xgb_grid= GridSearchCV(xgbf,
                        parameters,
                        cv = 2,
                        n_jobs = -1,
                        scoring='neg_mean_squared_error',
                        verbose=2)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
xgb_grid.fit(x,y)
submiss=xy_maker.sub_decoder(X_sub,xgb_grid.predict(X_sub))
submiss.index.name='id'

In [ ]:
submiss.to_csv('sub_amtonly_xgb.csv', encoding='utf-8-sig')

In [ ]:
model = LGBMRegressor( objective ='regression',
        num_leave = 1, learning_rate = 0.01, n_estimators=1000, max_depth = 3, boosting = 'gbdt',  feature_fraction =1, 
        max_bin = 80, tree_method='gpu_hist',  refit=True , random_state= 0 )